In [ ]:
#Import findings in tabular format
#Answer to question c of defined task

In [ ]:
import pandas as pd
import pandapower as pp

In [ ]:
# Initialize network
net = pp.create_empty_network()

In [ ]:
# Create high voltage buses (11 kV)
bus1 = pp.create_bus(net, vn_kv=11, name="Bus 1")
bus2 = pp.create_bus(net, vn_kv=11, name="Bus 2")

In [ ]:
# Create medium voltage buses (0.4 kV)
bus3 = pp.create_bus(net, vn_kv=0.4, name="Bus 3")
bus4 = pp.create_bus(net, vn_kv=0.4, name="Bus 4")

In [ ]:
# Low voltage buses connected to loads
bus5 = pp.create_bus(net, vn_kv=0.4, name="Bus 5")
bus6 = pp.create_bus(net, vn_kv=0.4, name="Bus 6")

In [ ]:
# External grid connections
pp.create_ext_grid(net, bus=bus1, vm_pu=1.0, name="Grid Connection 1")
pp.create_ext_grid(net, bus=bus2, vm_pu=1.0, name="Grid Connection 2")

In [ ]:
# Transformers with 11 kV to 0.4 kV ratio
pp.create_transformer_from_parameters(
    net, hv_bus=bus1, lv_bus=bus3, sn_mva=1, vn_hv_kv=11, vn_lv_kv=0.4, 
    vk_percent=6, vkr_percent=0.5, pfe_kw=0, i0_percent=0.1, name="Transformer 1"
)
pp.create_transformer_from_parameters(
    net, hv_bus=bus2, lv_bus=bus4, sn_mva=1, vn_hv_kv=11, vn_lv_kv=0.4, 
    vk_percent=6, vkr_percent=0.5, pfe_kw=0, i0_percent=0.1, name="Transformer 2"
)


In [ ]:
# High-voltage loads (connected to Bus 1 and Bus 2)
pp.create_load(net, bus=bus1, p_mw=0.8, q_mvar=0.4, name="HV Load 1")
pp.create_load(net, bus=bus2, p_mw=0.9, q_mvar=0.45, name="HV Load 2")

# Low-voltage loads (connected to Bus 5 and Bus 6)
pp.create_load(net, bus=bus5, p_mw=0.4, q_mvar=0.2, name="LV Load 3")
pp.create_load(net, bus=bus6, p_mw=0.3, q_mvar=0.15, name="LV Load 4")

# Lines between transformers and load buses
pp.create_line_from_parameters(
    net, from_bus=bus3, to_bus=bus5, length_km=0.1, r_ohm_per_km=0.1, 
    x_ohm_per_km=0.2, c_nf_per_km=0, max_i_ka=0.4, name="Line 1"
)
pp.create_line_from_parameters(
    net, from_bus=bus4, to_bus=bus6, length_km=0.1, r_ohm_per_km=0.1, 
    x_ohm_per_km=0.2, c_nf_per_km=0, max_i_ka=0.4, name="Line 2"
)


In [ ]:
# Switch to change topology between Bus 5 and Bus 6
switch = pp.create_switch(net, bus=bus5, element=bus6, et="b", closed=True, name="Topology Switch", numba=False)



In [ ]:
# Function to run load flow with switch open and closed, and export results
def run_loadflow_export(switch_closed):
    # Set the switch status and run the power flow
    net.switch.loc[switch, 'closed'] = switch_closed
    pp.runpp(net, numba=False)
    
    # Export bus voltages
    bus_voltages = net.res_bus[["vm_pu", "va_degree"]]
    bus_voltages.to_csv(f"bus_voltages_switch_{'closed' if switch_closed else 'open'}.csv")
    
    # Export transformer powers
    transformer_powers = net.res_trafo[["p_hv_mw", "q_hv_mvar", "p_lv_mw", "q_lv_mvar"]]
    transformer_powers.to_csv(f"transformer_powers_switch_{'closed' if switch_closed else 'open'}.csv")
    
    # Export line loadings (current as a percentage of max capacity)
    line_currents = net.res_line[["loading_percent"]]
    line_currents.to_csv(f"line_currents_switch_{'closed' if switch_closed else 'open'}.csv")
    
    print(f"Results exported for switch {'closed' if switch_closed else 'open'}")



In [ ]:
# Run load flows and export results
run_loadflow_export(switch_closed=True)  # Switch closed
run_loadflow_export(switch_closed=False) # Switch open
